In [42]:
using Plots
import Pkg; Pkg.add("DataFrames")
using DataFrames

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [43]:
function calc_angle(transduct_dis::Float64, diameter::Float64)
    theta_rad::Float64 = atan(diameter / transduct_dis)
    theta_degree::Float64 = rad2deg(theta_rad)
    L_dis::Float64 = sqrt(transduct_dis^2 + diameter^2)
    return [theta_degree, L_dis]
end

calc_angle (generic function with 1 method)

In [45]:
function calc_velocity(angle::Float64, L_dis::Float64, vel::Float64)
    c::Float64 = 1480.00
    td = L_dis / (c + vel * cosd(angle))
    tu = L_dis / (c - vel * cosd(angle))
    delta_time = tu - td
    v::Float64 = (L_dis * delta_time) / (2 * cosd(angle) * tu * td)
    return v
end

calc_velocity (generic function with 1 method)

In [46]:
function add_noise(value::Float64, noise_level::Float64)
    return value + (randn()*2-1) * noise_level
end

add_noise (generic function with 1 method)

In [103]:
# Função para calcular o perfil de velocidade parabólico
function parabolic_velocity_profile(r, R, vmax)
    return vmax * (1 - (r / R)^2)
end

# Função para calcular o tempo de trânsito usando o perfil parabólico
function calc_vel_parabolic(angle, L_dis, R, vmax)
    c = 1480.0
    num_points = 100
    rs = range(-R, stop=R, length=num_points)
    vs = parabolic_velocity_profile.(rs, R, vmax)
    avg_velocity = mean(vs)
    td = L_dis / (c + avg_velocity * cosd(angle))
    tu = L_dis / (c - avg_velocity * cosd(angle))
    delta_time = tu - td
    v::Float64 = (L_dis * delta_time) / (2 * cosd(angle) * tu * td)
    return [vmax,v]
end

calc_vel_parabolic (generic function with 1 method)

In [109]:
print(calc_vel_parabolic(45,10,0.5,5))

[5.0, 3.299663299663318]

In [113]:
velocitys = range(0.1, 10.0, length=20)
distance_btw_transductores = range(0.1, 50, length=20)  # cm
diameters = range(0.1, 10.0, length=20)  # cm
noise_level_time = 1e-6  # Nível de ruído para o tempo (segundos)
noise_level_velocity = 0.01  # Nível de ruído para a velocidade (m/s)

0.01